In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Maze/Maze

/content/drive/MyDrive/Maze/Maze


In [ ]:
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 6.8 MB/s 


In [ ]:
import os
import torch as T
import torch.nn as nn
import torch.optim as optim
import numpy as np
from read_maze import load_maze, get_local_maze_information
import pygame
import sys


pygame 2.1.2 (SDL 2.0.16, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [ ]:


SCREENSIZE = W, H = 1200, 800
mazeWH = 800
origin = ((W - mazeWH)/2, (H - mazeWH)/2)
lw = 2 # linewidth of maze-grid

# Colors
GREY = (140,140,140) # (15,15,15)
DARKGREY = (27, 27,0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
DARKGREEN = (0, 150, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 201)


class Canvas:
    def __init__(self):
        self.step_cntr = 0
        self.cntr = 0

        self.maze = load_maze()
        self.shape = 201

        pygame.init()
        self.surface = pygame.display.set_mode(SCREENSIZE)
        self.actor = (1,1)

    def drawSquareCell(self, x, y, dimX, dimY, col=(0, 0, 0)):
        pygame.draw.rect(
            self.surface, col,
            (x, y, dimX, dimY)
        )

    def drawSquareGrid(self, origin, gridWH):
        CONTAINER_WIDTH_HEIGHT = gridWH
        cont_x, cont_y = origin

        # DRAW Grid Border:
        # TOP lEFT TO RIGHT
        pygame.draw.line(
            self.surface, BLACK,
            (cont_x, cont_y),
            (CONTAINER_WIDTH_HEIGHT + cont_x, cont_y), lw)
        # # BOTTOM lEFT TO RIGHT
        pygame.draw.line(
            self.surface, BLACK,
            (cont_x, CONTAINER_WIDTH_HEIGHT + cont_y),
            (CONTAINER_WIDTH_HEIGHT + cont_x,
             CONTAINER_WIDTH_HEIGHT + cont_y), lw)
        # # LEFT TOP TO BOTTOM
        pygame.draw.line(
            self.surface, BLACK,
            (cont_x, cont_y),
            (cont_x, cont_y + CONTAINER_WIDTH_HEIGHT), lw)
        # # RIGHT TOP TO BOTTOM
        pygame.draw.line(
            self.surface, BLACK,
            (CONTAINER_WIDTH_HEIGHT + cont_x, cont_y),
            (CONTAINER_WIDTH_HEIGHT + cont_x,
             CONTAINER_WIDTH_HEIGHT + cont_y), lw)


    def placeCells(self):
        # GET CELL DIMENSIONS...
        cellBorder = 0
        celldimX = celldimY = (mazeWH / self.shape)

        # DOUBLE LOOP
        for rows in range(201):
            for cols in range(201):
                # Is the grid cell tiled ?
                if (self.maze[rows][cols] == 0):
                    self.drawSquareCell(
                        origin[0] + (celldimY * rows)
                        + cellBorder + lw / 2,
                        origin[1] + (celldimX * cols)
                        + cellBorder + lw / 2,
                        celldimX, celldimY, col=BLACK)
                if cols == 199 and rows == 199:
                    self.drawSquareCell(
                        origin[0] + (celldimY * rows)
                        + cellBorder + lw / 2,
                        origin[1] + (celldimX * cols)
                        + cellBorder + lw / 2,
                        celldimX, celldimY, col=BLUE)

    def step(self, visible, idx, path):
        """Run the pygame environment for displaying the maze structure and visible (local) environment of actor
        """
        self.get_event()
        self.set_visible(visible, idx, path)

        self.surface.fill(GREY)
        self.drawSquareGrid(origin, mazeWH)

        self.placeCells()
        self.draw_visible()
        pygame.display.update()
        self.step_cntr += 1

    def set_visible(self, visible, idx, path):
        self.vis = visible
        self.actor = idx
        self.path = path

    def draw_visible(self):
        """Draw the visible environment around the actor

        Notes
        -----
        RED - signifies a fire
        DARKGREY - signifies a visible wall
        WHITE - signifies a path
        GREEN - indicates the actor's position
        """
        celldimX = celldimY = (mazeWH / self.shape)

        #self.visible = self.vis
        for s in self.path:
            self.drawSquareCell(
                origin[0] + (celldimY * s[1])
                + lw / 2,
                origin[1] + (celldimX * s[0])
                + lw / 2,
                celldimX, celldimY, col=DARKGREEN)


        for row in range(3):
            for col in range(3):
                c = self.actor[0] + (col - 1)
                r = self.actor[1] + (row - 1)


                if row == 1 and col == 1:
                    self.drawSquareCell(
                        origin[0] + (celldimY * r)
                        + lw / 2,
                        origin[1] + (celldimX * c)
                        + lw / 2,
                        celldimX, celldimY, col=GREEN)
                else:
                    if self.vis[row][col][1] > 0:

                        self.drawSquareCell(
                            origin[0] + (celldimY * r)
                            + lw / 2,
                            origin[1] + (celldimX * c)
                            + lw / 2,
                            celldimX, celldimY, col=RED)
                    # else:
                    #     if self.vis[row][col][0] == 1:
                    #         self.drawSquareCell(
                    #             origin[0] + (celldimY * r)
                    #             + lw / 2,
                    #             origin[1] + (celldimX * c)
                    #             + lw / 2,
                    #             celldimX, celldimY, col=WHITE)
                    #     elif self.vis[row][col][0] == 0:
                    #         self.drawSquareCell(
                    #             origin[0] + (celldimY * r)
                    #             + lw / 2,
                    #             origin[1] + (celldimX * c)
                    #             + lw / 2,
                    #             celldimX, celldimY, col=DARKGREY)

    def get_event(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()

In [ ]:
'''
代码的来源
https://github.com/alvaroprat97/RL-Maze-Solver/blob/master/Complex_Maze/agent.py
https://github.com/luispsantos/EL2805-Reinforcement-Learning/blob/main/lab2/lab2_instructions.pdf
https://zhuanlan.zhihu.com/p/346165057
https://github.com/xiaoyw71/Reinforcement-learning-practice
https://github.com/luispsantos/EL2805-Reinforcement-Learning/blob/4117514ae3cb3003df3785db64a9dc1d3a2b6954/lab1/problem1/maze.py
Source - https://github.com/philtabor/Youtube-Code-Repository/blob/master/ReinforcementLearning/CombinedExperienceReplay/memory_solution.py
https://github.com/lukun199/DQN_maze/blob/main/Main.py
'''
import matplotlib.pyplot as plt
import time
from IPython import display
global action_dir
action_dir = {"0": {"id":'stay',
                    "move":(0,0)},
              "1": {"id":'up',
                    "move":(0,-1)},
              "2": {"id":'left',
                    "move":(-1,0)},
              "3": {"id":"down",
                    "move":(0,1)},
              "4": {"id":'right',
                    "move":(1,0)}
              }

global rewards_dir
rewards_dir = {"onwards": -.04,
              "backwards":-0.0,
              "visited":-0.3,
              "blockedin":-0.1,
              "fire":-1.,
              "wall":-.75,
              "stay":-0.3,
              }

LIGHT_RED = '#FFC4CC'
LIGHT_GREEN = '#95FD99'
BLACK = '#FFFFFF'
WHITE = '#000000'
LIGHT_PURPLE = '#E8D0FF'
LIGHT_ORANGE = '#FAE0C3'


class Qmaze:
    def __init__(self):
        # Reset the total number of steps which the agent has taken
        self.num_steps_taken = 0
        
        self.rat = (1,1)
        self.goal = (199,199)
        self.rat_path = [self.rat]
        self.around_info = []
        self.observation = self.observe_environment

        
    @property
    def reset(self):
        self.num_steps_taken = 0
        
        self.rat = (1,1)
        self.goal = (199,199)
        self.rat_path = [self.rat]
        self.around_info = []
        self.observation = self.observe_environment


    @property
    def get_loc_info(self):
        return self.observation

    @property
    def get_rat_pos(self):
        return self.rat

    @property
    def get_rat_path(self):
        return self.rat_path

    @property
    def get_around_info(self):
      col, rol = self.get_rat_pos
      location = get_local_maze_information(rol,col)
      self.around_info = location.copy()
      return self.around_info

    # def get_around_info(self):
    #   return self.around_info
    @property
    def observe_environment(self):
        col, rol = self.get_rat_pos
        location = get_local_maze_information(rol,col)
        # self.set_around_info
        temp = []

        for i in range(location.shape[0]):
            for j in range(location.shape[1]):
                if location[i][j][0] == 0:
                    temp.append(0) #wall
                elif location[i][j][0] == 1 and location[i][j][1] ==0:
                    temp.append(1) #no fire and empty
                elif location[i][j][0] == 1 and location[i][j][1] >0:
                    temp.append(location[i][j][1]) #fire
                else:
                    temp.append(1) #自己的位置

        self.rat_path.append(self.rat)
        self.observation = temp
        return self.observation

    
    def animate_solution(self):
      maze = load_maze()
      path = self.get_rat_path
    # Map a color to each cell in the maze
      col_map = {0: WHITE, 1: BLACK, 2: LIGHT_GREEN, -6: LIGHT_RED, -1: LIGHT_RED}

    # Size of the maze
      rows, cols = maze.shape

    # Create figure of the size of the maze
      fig = plt.figure(1, figsize=(cols, rows))

    # Remove the axis ticks and add title title
      ax = plt.gca()
      ax.set_title('Policy simulation')
      ax.set_xticks([])
      ax.set_yticks([])

    # Give a color to each cell
      colored_maze = [[col_map[maze[j, i]]
                     for i in range(cols)] for j in range(rows)]

    # Create figure of the size of the maze
      fig = plt.figure(1, figsize=(cols, rows))

    # Create a table to color
      grid = plt.table(cellText=None,
                     cellColours=colored_maze,
                     cellLoc='center',
                     loc=(0, 0),
                     edges='closed')

    # Modify the hight and width of the cells in the table
      tc = grid.properties()['children']
      for cell in tc:
          cell.set_height(1.0/rows)
          cell.set_width(1.0/cols)

    # Update the color at each frame
      for i in range(len(path)):

        ax.set_title(f'\t \t \t \t  Policy simulation \t \t \t t {i} T {len(path)-1}'.expandtabs())
        # First clear the prev illustration, if path[i] is same as path[i-1] then it is already changed!
        # Illustration of current status
        if i > 0:
        # if path[i][0:2] != path[i-1][0:2]:
            grid.get_celld()[(path[i-1])].set_facecolor(col_map[maze[path[i-1]]])


        # Agent illustration
        grid.get_celld()[(path[i])].set_facecolor(LIGHT_ORANGE)
        grid.get_celld()[(path[i])].get_text().set_text('Rat')



        # Position is the same and it is WIN!
        # if maze[path[i][0:2]] == 2:
        #     grid.get_celld()[(path[i][0:2])].set_facecolor(LIGHT_GREEN)
        #     grid.get_celld()[(path[i][0:2])].get_text().set_text('WIN')
        #     break # Since nothing changes

        display.display(fig)
        display.clear_output(wait=True)
        time.sleep(1)

    def step(self, action, score):
        """Sample environment dependant on action which has occurred

        Action Space
        ------------
        0 - no move
        1 - up
        2 - left
        3 - down
        4 - right

        """
        #time.sleep(1) # delay for time animation
        self.num_steps_taken += 1 # increment time

        global action_dir # Fetch action directory containing the properties of each action w.r.t environment
        act_key = str(action)
        global rewards_dir # Fetch reward directory

        x_inc, y_inc = action_dir[act_key]['move'] # fetch movement from position (1,1)

        # If too much time elapsed you die in maze :( (terminate maze at this point)
        if score < -100:
            print('I became an old man and dies in this maze...')
            return self.observe_environment, -1., True # terminate

        obsv_mat = self.get_around_info # get prior position
        x, y = self.rat

        x_loc, y_loc = (1 + x_inc, 1 + y_inc) # Update Local Position

        is_blocked = True
        for o in obsv_mat:
            for s in o:
                if s[0] == 1 and s[1] == 0: # if there is a path and no fire, we are not blocked
                    is_blocked = False

        if action_dir[act_key]['id'] == 'stay' and is_blocked: # if we need to stay then reward
            return self.observe_environment, rewards_dir['blockedin'], False
        elif action_dir[act_key]['id'] == 'stay': # if we stay for no reason then penalise
            return self.observe_environment, rewards_dir['stay'], False

        if obsv_mat[y_loc][x_loc][0] == 0: # check for a wall
            return self.observe_environment, rewards_dir['wall'], False
        if obsv_mat[y_loc][x_loc][1] > 0: # check for a wall
            return self.observe_environment, rewards_dir['fire'], True

        # So if we do successfully move
        self.rat = new_pos = (x + x_inc, y + y_inc) # new global position if we move into a free space
        # Have we reached the end?
        if new_pos == (199, 199):
            return self.observation, rewards_dir['end'], True

        # Have we visited this spot already?
        if self.rat in self.rat_path:
            return self.observe_environment, rewards_dir['visited'], False

        # Are we moving towars the goal?
        if x_inc > 0 or y_inc > 0:
            return self.observe_environment, rewards_dir['onwards'], False

        # finally our only choice is to move away from goal
        return self.observe_environment, rewards_dir['backwards'], False




In [ ]:
class ReplayBuffer():
    def __init__(self, input_shape, max_size, batch_size):
        self.batch_size = batch_size
        self.mem_size = max_size # bound memory so we don't crash RAM
        self.mem_cntr = 0 # simulate stack by knowing whee in memory we are

        # initialise state memory with 0 values
        self.state_mem = np.zeros((self.mem_size, *input_shape), dtype=np.float32)
        self.state_mem_ = np.zeros((self.mem_size, *input_shape), dtype=np.float32)

        self.action_mem = np.zeros(self.mem_size, dtype=np.int64)
        self.reward_mem = np.zeros(self.mem_size, dtype=np.float32)
        # self.terminal_mem = np.zeros(self.mem_size, dtype=bool)

    def store_buffer(self, state, state_, reward, action):
        # wrap counter to know where we are in memory
        idx = self.mem_cntr % self.mem_size
        # assign states, reward and action to memory
        self.state_mem[idx] = state
        self.state_mem_[idx] = state_
        self.reward_mem[idx] = reward
        self.action_mem[idx] = action
        # increment position in memory
        self.mem_cntr += 1

    def sample_buffer(self):
        # Max existing memory size (if mem not full set max mem to counter value)
        max_mem = min(self.mem_size, self.mem_cntr)
        btch = np.random.choice(max_mem, self.batch_size, replace=False)

        states = self.state_mem[btch]
        states_ = self.state_mem_[btch]
        actions = self.action_mem[btch]
        rewards = self.reward_mem[btch]

        return states, actions, rewards, states_

    def is_sufficient(self):
        return self.mem_cntr > self.batch_size

In [ ]:
class Network(nn.Module):
    # The class initialisation function. This takes as arguments the dimension of the network's input (i.e. the dimension of the state), and the dimension of the network's output (i.e. the dimension of the action).
    def __init__(self, input_dims, n_actions, lr):
        # Call the initialisation function of the parent class.
        super(Network, self).__init__()
        # Define the network layers. This example network has two hidden layers, each with 100 units.
        self.layer_1 = T.nn.Linear(input_dims, 256)
        self.layer_2 = T.nn.Linear(256, 256)
        self.output_layer = T.nn.Linear(256,n_actions)

        self.optimiser = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    # Function which sends some input data through the network and returns the network's output. In this example, a ReLU activation function is used for both hidden layers, but the output layer has no activation function (it is just a linear layer).
    def forward(self, input):
        layer_1_output = T.nn.functional.relu(self.layer_1(input))
        layer_2_output = T.nn.functional.relu(self.layer_2(layer_1_output))
        action = self.output_layer(layer_2_output)
        return action


In [ ]:

class Agent():
    def __init__(self, gamma, epsilon, buffer_size, input_dims, n_actions, lr, mem_size, batch_size, eps_min=0.01, eps_dec=5e-7, replace=10):
        
        self.learn_step_counter = 0
        self.gamma = gamma
        self.epsilon = epsilon
        self.lr = lr
        self.n_actions = n_actions
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.eps_min = eps_min
        self.eps_dec = eps_dec
        self.replace_target_thresh = replace
        self.action_space = [i for i in range(self.n_actions)]
        self.buffer_size = buffer_size
        self.memory = ReplayBuffer(buffer_size, mem_size, batch_size)
        self.q_eval = Network(self.input_dims, self.n_actions, self.lr)
        self.q_next = Network(self.input_dims, self.n_actions, self.lr)

    def greedy_epsilon(self, observation):
        # if we randomly choose max expected reward action
        if np.random.random() > self.epsilon:
            actions = self.q_eval.forward(T.tensor([observation], dtype=T.float).to(self.q_eval.device))
            action = T.argmax(actions).item()
        # otherwise random action
        else:
            action = np.random.choice(self.action_space)
        return action

    
    def store_transition(self, state, state_, reward, action):
        self.memory.store_buffer(state, state_, reward, action)

    def replace_target_network(self):
      dqn_dict =  T.nn.Module.state_dict(self.q_eval)
      T.nn.Module.load_state_dict(self.q_next,dqn_dict)
      

    def dec_epsilon(self):
        self.epsilon = self.epsilon - self.eps_dec \
            if self.epsilon > self.eps_min else self.eps_min


    def learn(self):
        # Wait for memory to fill up before learning from empty set
        if not self.memory.is_sufficient():
            return

        # Start AD
        self.q_eval.optimiser.zero_grad()
        self.replace_target_network()

        # sample memory
        state, actions, reward, state_ = self.memory.sample_buffer()
        idxs = np.arange(self.batch_size) # need for array slicing later

        state_btch = T.tensor(state).to(self.q_eval.device)
        action_btch = T.tensor(actions).to(self.q_eval.device)
        rewards_btch = T.tensor(reward).to(self.q_eval.device)
        state_btch_ = T.tensor(state_).to(self.q_eval.device)

        q_pred = self.q_eval.forward(state_btch)[idxs, actions]
        q_next = self.q_next.forward(state_btch_)
        q_eval = self.q_eval.forward(state_btch_)

        max_actions = T.argmax(q_eval, dim=1)

        # # apply mask for terminates networks
        # q_next[{}] = 0.0

        q_target = rewards_btch + self.gamma * q_next[idxs, max_actions]

        loss = self.q_eval.loss(q_target, q_pred).to(self.q_eval.device)
        loss.backward()

        self.q_eval.optimiser.step()
        self.learn_step_counter += 1
        self.dec_epsilon()

In [ ]:


def run(train_chck=True, chckpt=False, lr=0.01, epsilon=0.9,
        gamma=0.9, episodes=100, netname='default.pt'):
    # Default (Fixed) Parameters
    epsilon_min = 0.1
    epsilon_dec = 1e-6
    size = [9]
    input_dims = 9
    output_dims = 5

    replace_testnet = 50
    memsize = 1000000 # https://arxiv.org/abs/1712.01275
    batch_size = 64

    agent = Agent(gamma=gamma, epsilon=epsilon, lr=lr,
                  input_dims=input_dims,buffer_size= size, n_actions=output_dims, mem_size=memsize, eps_min=epsilon_min,
                  batch_size=batch_size, eps_dec=epsilon_dec, replace=replace_testnet)

    if not train_chck:
        canv = Canvas()
    else:
        maze = load_maze()

    # if chckpt:
    #     agent.load_models()

    # plt = Plotter()
    env = Qmaze()
    print('...starting...')
    env.reset
    for i in range(episodes):
        
        done = False
        
        observation = env.observe_environment

        if not train_chck:
            canv.set_visible(env.get_around_info.copy(), env.get_rat_pos, [])

        score = 0
        while not done:
            canv.step(env.around_info.copy(), env.rat, env.rat_path)
            print("position:",env.rat_path)
            action = agent.greedy_epsilon(observation)
            print("action:",action)
            observation_, reward, done = env.step(action, score)
            score += reward
            print("score:",score)
            agent.store_transition(observation, observation_, reward, action)
            agent.learn()
            observation = observation_

            
            # print(done)

        if env.rat==env.goal:
          print("sucessful")
          break
        else:
          # env.animate_solution()
          print("-----------------Next iteration-------------------")
    
    # env.animate_solution()
 
        # plt.data_in(score, wall_cntr=env.wall_cntr, stay_cntr=env.stay_cntr, visit_cntr=env.visit_cntr)
        # print(f'Ep {i}, score {score}, avg {plt.scores_avg[-1]}, epsilon {agent.epsilon}, lr {lr}')
        # print(f'    Stayed {env.stay_cntr} : Walls {env.wall_cntr}')
        # Save NN every 10 its
        # if i > 10 and i % 10 == 0:
        #     agent.save_models()
            # plt.live_plot()

if __name__ == '__main__':
    run(train_chck=False, chckpt=True, episodes=1000, lr=0.0001, epsilon=0.5, gamma=0.90)


...starting...
position: [(1, 1), (1, 1), (1, 1)]
action: 1
score: -0.75
position: [(1, 1), (1, 1), (1, 1), (1, 1)]
action: 3
score: -0.79
position: [(1, 1), (1, 1), (1, 1), (1, 1), (1, 2)]
action: 1
score: -1.09
position: [(1, 1), (1, 1), (1, 1), (1, 1), (1, 2), (1, 1)]
action: 1
score: -1.84
position: [(1, 1), (1, 1), (1, 1), (1, 1), (1, 2), (1, 1), (1, 1)]
action: 4
score: -1.8800000000000001
position: [(1, 1), (1, 1), (1, 1), (1, 1), (1, 2), (1, 1), (1, 1), (2, 1)]
action: 3
score: -2.63
position: [(1, 1), (1, 1), (1, 1), (1, 1), (1, 2), (1, 1), (1, 1), (2, 1), (2, 1)]
action: 2
score: -2.9299999999999997
position: [(1, 1), (1, 1), (1, 1), (1, 1), (1, 2), (1, 1), (1, 1), (2, 1), (2, 1), (1, 1)]
action: 1
score: -3.6799999999999997
position: [(1, 1), (1, 1), (1, 1), (1, 1), (1, 2), (1, 1), (1, 1), (2, 1), (2, 1), (1, 1), (1, 1)]
action: 1
score: -4.43
position: [(1, 1), (1, 1), (1, 1), (1, 1), (1, 2), (1, 1), (1, 1), (2, 1), (2, 1), (1, 1), (1, 1), (1, 1)]
action: 0
score: -4.729999

KeyboardInterrupt: ignored